In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install numpy 
!pip install pandas
!pip install sklearn
!pip install torch 
!pip install sklearn
!pip install seaborn
!pip install matplotlib
!pip install json
!pip install fast_ml

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 880 kB 71.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.1 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0e58dd3f29cdf359d0260483bec6b4f7e8db6c1826e90776bb25d45c1aaa7080
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     |████████████████████████████████| 42 kB 656 kB/s 


In [3]:
import numpy as np #Für Matrizen
import pandas as pd #Für Dataframes
import re
from typing import List, Any, Set, Dict, Tuple, Optional
from ast import literal_eval
import json # load json module
from sklearn.model_selection import train_test_split
import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
from transformers import BertTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification
from fast_ml.model_development import train_valid_test_split

In [4]:
dict_name = "l_dict_m1"
json_dir = f'/content/drive/MyDrive/{dict_name}.json'

In [5]:
with open(json_dir) as jsonFile:
    labels_Input = json.load(jsonFile)
    jsonFile.close()
print(max(labels_Input), min(labels_Input), len(labels_Input))
print(labels_Input)

SNOMED CT NOT SNOMED CT 2
{'SNOMED CT': {'name': '', 'synonym': '', 'code': 'SNOMED CT', 'index': 0}, 'NOT SNOMED CT': {'name': '', 'synonym': '', 'code': 'NOT SNOMED CT', 'index': 1}}


In [6]:
labels_Output = {}
for k, v in labels_Input.items():
    labels_Output[v["index"]] = {
            "name": v["name"],
            "code": k,
            "index": v["index"]
            } 
print(len(labels_Output))

2


**Trainingsdaten vorbereiten**

In [7]:
excel_path = '/content/drive/MyDrive/train_data.xlsx'
sheet_name = "labeled_dataset1"

In [8]:
df = pd.read_excel(excel_path, sheet_name, index_col=None)
df = df.astype({'Label': str})
print(df)
print(len(df))

                                                     Text      Label
0                          Excision of lesion of patella   SNOMED CT
1                           Excision of lesion of patella  SNOMED CT
2           Local excision of lesion or tissue of patella  SNOMED CT
3                   Structure of posterior carpal region   SNOMED CT
4                                 Posterior carpal region  SNOMED CT
...                                                   ...        ...
464496  intestinal continuity according to Hartmann\n1...  SNOMED CT
464497  intestinal continuity according to Hartmann\n1...  SNOMED CT
464498  intestinal continuity according to Hartmann\n1...  SNOMED CT
464499  intestinal continuity according to Hartmann\n1...  SNOMED CT
464500  intestinal continuity according to Hartmann\n1...  SNOMED CT

[464501 rows x 2 columns]
464501


In [9]:
df['Label'].value_counts()

SNOMED CT        241371
NOT SNOMED CT    223130
Name: Label, dtype: int64

In [10]:
label_code = []
code_fails = []
for index, row in df.iterrows():
    if row["Label"] in labels_Input:
        label_code.append(labels_Input[row["Label"]]["index"])
    else:
        label_code.append(labels_Input["-1"]["index"])

df['label_code'] = label_code
print(df)

                                                     Text      Label  \
0                          Excision of lesion of patella   SNOMED CT   
1                           Excision of lesion of patella  SNOMED CT   
2           Local excision of lesion or tissue of patella  SNOMED CT   
3                   Structure of posterior carpal region   SNOMED CT   
4                                 Posterior carpal region  SNOMED CT   
...                                                   ...        ...   
464496  intestinal continuity according to Hartmann\n1...  SNOMED CT   
464497  intestinal continuity according to Hartmann\n1...  SNOMED CT   
464498  intestinal continuity according to Hartmann\n1...  SNOMED CT   
464499  intestinal continuity according to Hartmann\n1...  SNOMED CT   
464500  intestinal continuity according to Hartmann\n1...  SNOMED CT   

        label_code  
0                0  
1                0  
2                0  
3                0  
4                0  
...      

In [11]:
labels_Output = {}
for k, v in labels_Input.items():
    labels_Output[v["index"]] = {
      "name": v["name"],
      "code": k,
      "index": v["index"]
  } 

print(min(labels_Output), max(labels_Output), len(labels_Output))

0 1 2


In [12]:
df['label_code'].value_counts()

0    241371
1    223130
Name: label_code, dtype: int64

In [13]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'label_code', 
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train.index.values, 'data_type'] = 'train'
df.loc[X_valid.index.values, 'data_type'] = 'val'
df.loc[X_test.index.values, 'data_type'] = 'test'

df.groupby(['Label', 'label_code', 'data_type']).count()

Text
Label         label_code data_type        
NOT SNOMED CT 1          test        22267
                         train      178745
                         val         22118
SNOMED CT     0          test        24184
                         train      192855
                         val         24332

**Training vorbereiten**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

print("Encode Train Data.......")
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)
print("Encode Validation Data.......")
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)
print("Encode Test Data.......")
encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label_code.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label_code.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label_code.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Encode Train Data.......
Encode Validation Data.......
Encode Test Data.......


In [ ]:
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2',
                                                      num_labels=len(labels_Input),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import Adam

optimizer = Adam(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs =  5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
import random
import torch
import numpy as np

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        #print(loss)
        logits = outputs[1]
        #print(logits)
        loss_val_total += loss.item()
        #print(loss_val_total)

        logits = logits.detach().cpu().numpy()
        #print(logits)
        label_ids = inputs['labels'].cpu().numpy()
        #print(label_ids)

        #print(label_ids)
        predictions.append(logits)
        true_vals.append(label_ids)


    loss_val_avg = loss_val_total/len(dataloader_val) 
    #print(predictions)
    #print(true_vals)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    #print(len(predictions))
    #print(len(true_vals))

            
    return loss_val_avg, predictions, true_vals

In [ ]:
import random
import torch
import numpy as np

def test(dataloader_test):

    model.eval()
    
    predictions, true_vals = [], []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)     

        logits = outputs[1]

        logits = logits.detach().cpu().numpy()

        label_ids = inputs['labels'].cpu().numpy()

        predictions.append(logits)
        true_vals.append(label_ids)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return predictions, true_vals

**Training durchführen**

In [ ]:
training_losses = []
valid_losses = []
fOnes = []
# set initial loss to infinite
best_valid_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    training_losses.append(loss_train_avg)
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    valid_losses.append(val_loss)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/saved_weights_M1_v1.model')  
        
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    fOnes.append(val_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/46450 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt
# line 1 points
x1 = range(len(training_losses))
y1 = training_losses
# plotting the line 1 points 
plt.plot(x1, y1, label = "line 1")
# line 2 points
x2 = range(len(valid_losses))
y2 = valid_losses
# plotting the line 2 points 
plt.plot(x2, y2, label = "line 2")
plt.xlabel('Epochen')
# Set the y axis label of the current axis.
plt.ylabel('Training')
# Set a title of the current axes.
plt.title('Losses')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

In [ ]:
# line 1 points
x1 = range(len(fOnes))
y1 = fOnes
# plotting the line 1 points 
plt.plot(x1, y1, label = "line 1")
plt.xlabel('Epochen')
# Set the y axis label of the current axis.
plt.ylabel('F1')
# Set a title of the current axes.
plt.title('F1 Validation')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()

**Testen**

In [ ]:
bert_base_model = "dmis-lab/biobert-base-cased-v1.2"
training_state = '/content/drive/MyDrive/saved_weights_M1_v1.model'

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model = BertForSequenceClassification.from_pretrained(bert_base_model,
                                                      num_labels=len(labels_Input),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
print("BFSC as Model loaded....")

model.load_state_dict(torch.load(training_state, map_location=device))
print("state_dict loaded.....")

#tokenizer = BertTokenizer.from_pretrained(bert_base_model, 
                                          #output_hidden_states=False,
                                          #map_location=device)
#print("Tokenizer loaded.....")

In [ ]:
predictions, true_vals = test(dataloader_test)

In [ ]:
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
preds = np.argmax(predictions, axis = 1)
print(classification_report(true_vals, preds))

In [ ]:
sentence = "Comprehensive care of lower urinary tract infections"
model.eval()

input = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors = "pt")
output = model(**input)
logits = output.logits
soft = torch.nn.Softmax(dim=0)
soft_tensor = soft(logits[0])
probs = soft_tensor.detach().numpy()
#print(probs)
probabilities = {}
for k, v in labels_Input.items():
    probabilities[k] = probs[v["index"]]
print(probabilities)
print("-----------------------------------------------------------------------------------------")
